# Install

In [ ]:
import os

# check if `pytorch-CycleGAN-and-pix2pix` is already cloned
if not os.path.exists('pix2pixHD'):
	!git clone https://github.com/NVIDIA/pix2pixHD.git
else:
    print('pix2pixHD already cloned')

In [ ]:
os.chdir('pix2pixHD')

In [ ]:
!pwd

In [ ]:
%pip install -q dominate

# Checkpoints

In [ ]:
if not os.path.exists('checkpoints/fish_pix2pix'):
    !bash ../scripts/download_pix2pixHD_pretrained.sh
else:
    print('Pretrained model already exists')

# Check Testing Dataset

In [ ]:
import os

try:
    os.chdir('../dataset')
    if not os.path.exists('test_pose'):
        !bash ../scripts/download_test_pose.sh
    else:
        print('Test pose already exists')
except:
    print('Error: dataset folder not found')


In [ ]:
# move the folder to model folder
import shutil

try:
    os.chdir('..')
except:
    print('Error: Already in the root folder')


source_folder = './dataset/test_pose'
target_folder = './pix2pixHD/datasets/test_pose'

if not os.path.exists(target_folder):
    shutil.move(source_folder, target_folder)
else:
    print('Test pose already exists in pix2pixHD folder')

In [ ]:
# change os path
os.chdir('pix2pixHD')

In [ ]:
!pwd

# Testing

In [ ]:
!python test.py --dataroot datasets/test_pose --name fish_pix2pix --label_nc 0 --no_instance --netG global --resize_or_crop none --how_many 100